In [1]:
import requests
import json
from bs4 import BeautifulSoup
import sqlite3
import string
import collections

In [2]:
dbs = [
    'C:\\Users\\mag\\Documents\\Python-2019\\db\\sberbank_hh.db',
    'C:\\Users\\mag\\Documents\\Python-2019\\db\\drweb_hh.db',
    'C:\\Users\\mag\\Documents\\Python-2019\\db\\ifmo_hh.db',
    'C:\\Users\\mag\\Documents\\Python-2019\\db\\kasper_hh.db',
    'C:\\Users\\mag\\Documents\\Python-2019\\db\\tinkoff_hh.db'
]

### Обработка баз данных
    1. прочитать из .db файла
    2. удалить дубликаты по совпадающему id
    3. проверить с помощью api profarea_id
    
    из sberbank_hh было отобрано: 378 вакансий
    из drweb_hh было отобрано: 134 вакансии
    из ifmo_hh было отобрано: 40 вакансий
    из kasper_hh было отобрано: 510 вакансий
    из tinkoff_hh было отобрано: 175 вакансий
    

#### 1 шаг - чтение

In [166]:
new_db = dbs[5]

conn = sqlite3.connect('C:\\Users\\mag\\Documents\\Python-2019\\unecon-lab-assignments-master(03.12)\\coursework (trends analysis)\\final.db')
cursor = conn.cursor()

cursor.execute('SELECT * from vacancy')
vacancies = cursor.fetchall()
conn.close()

In [168]:
vacancies[0]

('29654783',
 '<div class="g-user-content" data-qa="vacancy-description" itemprop="description"><p><strong>Обязанности:</strong></p> <p>• Обработка/решение поступающих заявок, консультирование сотрудников банка по вопросам технической поддержки автоматизированных рабочих мест пользователей (ПК/Оргтехника/Мобильные устройства).</p> <p>• Удаленное решение заявок по установке/настройке ПО средствами SCCM и с использованием средств удаленного администрирования (RDP).</p> <p>• Взаимодействие с сервисными компаниями по вопросам оказания услуги тех.поддержки.</p> <p><strong>Требования:</strong></p> <p>• Законченное высшее образование по направлению Информационные технологии, возможно последний курс.</p> <p>• Желателен опыт работы в сфере ИТ.</p> <p>• Экспертное знание MS Windows 7 и MS Office 2010 и выше.</p> <p>• Опыт поддержки ПК, системного ПО, в т.ч. навыки удаленного администрирования ПК.</p> <p>• Знание технического устройства ПК и его аппаратных составляющих.</p> <p>• Стрессоустойчивос

In [89]:
vacancies[0]

(None,
 'https://hh.ru/vacancy/26740968',
 'Начинающий специалист по залоговому кредитованию',
 '<p>Вакансия компании: Тинькофф</p><p>Создана: 15.01.2019</p><p>Регион: Москва</p><p>Предполагаемый уровень месячного дохода: от\xa055\xa0000\xa0руб.</p>',
 '2019-01-15T09:56:55.656+03:00',
 '<!DOCTYPE html>\n<html class="desktop"><!--request: 154753778643054fa3a6eecdf2829496--><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><title>Вакансия Начинающий специалист по залоговому кредитованию в Москве, работа в Тинькофф</title><meta name="description" content="Вакансия Начинающий специалист по залоговому кредитованию. Зарплата: от 55000 руб.. Москва. Требуемый опыт: не требуется. Полная занятость. Дата публикации: 15.01.2019."><meta name="keywords" content="Вакансия Начинающий специалист по залоговому кредитованию в Москве, работа в Тинькофф"><meta property="og:image" content="https://spb.hh.ru/employer-logo/2848221.pn

#### 2 шаг - удаление дубликатов

In [5]:
def get_descr_local(raw_html):
    soup = BeautifulSoup(raw_html, 'html.parser')
    return soup.find('div', class_='g-user-content')

In [6]:
def get_descr_api(id_):
    req = requests.get('https://api.hh.ru/vacancies/' + id_).json()
    try:
        desc = req['description']
        return desc
    except:
        return None

In [90]:
ids = []
vacs_clean = {}
left = len(vacancies)
for job in vacancies:
    id_ = job[1].split('/')[-1]
    if id_ == '':
        id_ = job[1].split('/')[-2]    
    if id_ not in ids:
        ids.append(id_)
        desc = get_descr_local(job[5])
        if desc == None:
            desc = get_descr_api(id_)
        vacs_clean[id_] = desc
    left -= 1
    if left % 100 == 0 and left != 0:
        print(f'осталось обработать {left}/{len(vacancies)} вакансий.')

In [91]:
len(vacs_clean)

1501

In [92]:
list(vacs_clean.items())

[('26740968',
  '<p>Tinkoff запустил новый проект и ищет специалистов в отдел по работе с залогами.</p> <ul> <li>Работа в нашем проекте дает Вам возможность быстрого карьерного роста;</li> <li>Например, сейчас Вы придете на вакансию специалиста, а уже через год или раньше станете руководителем группы;</li> <li>У Вас сейчас есть уникальный шанс быть в одной команде с нами!</li> </ul> <p><strong>Обязанности:</strong></p> <ul> <li>Звонить клиентам, которым банк одобрил кредит под залог;</li> <li>Согласовывать условия кредита;</li> <li>Консультировать клиентов по их вопросам;</li> <li>Назначать встречи с представителем банка.</li> </ul> <strong>Условия:</strong> <ul> <li>График работы 2/2 с плавающими выходными;</li> <li>Оклад + ежемесячная неограниченная премия;</li> <li>Место работы в шаговой доступности от м. Марьина Роща;</li> <li>Стипендия за обучение (в случае его прохождения и работы в штате более 2 месяцев);</li> <li>Скидка на посещение фитнес-клуба.</li> </ul> <strong>Возможности:

#### 3 шаг - фильтрация по сфере деятельности

In [93]:
filtered = {}
left = len(vacs_clean)
for id_ in vacs_clean:
    req = requests.get('https://api.hh.ru/vacancies/' + id_).json()
    try:
        profarea = req['specializations'][0]['profarea_id']
        if profarea == '1':
            filtered[id_] = vacs_clean[id_]
    except:
        pass
    left -= 1
    if left % 100 == 0 and left != 0:
        print(f'осталось обработать {left}/{len(vacs_clean)} вакансий.')

осталось обработать 1500/1501 вакансий.
осталось обработать 1400/1501 вакансий.
осталось обработать 1300/1501 вакансий.
осталось обработать 1200/1501 вакансий.
осталось обработать 1100/1501 вакансий.
осталось обработать 1000/1501 вакансий.
осталось обработать 900/1501 вакансий.
осталось обработать 800/1501 вакансий.
осталось обработать 700/1501 вакансий.
осталось обработать 600/1501 вакансий.
осталось обработать 500/1501 вакансий.
осталось обработать 400/1501 вакансий.
осталось обработать 300/1501 вакансий.
осталось обработать 200/1501 вакансий.
осталось обработать 100/1501 вакансий.


In [94]:
len(filtered)

175

In [95]:
#all_filtered = {}
all_filtered.update(filtered)

In [119]:
len(all_filtered)

1237

In [112]:
saved = {id_:str(all_filtered[id_]) for id_ in all_filtered}